In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from tqdm.notebook import tqdm
import glob
import numpy as np
from utils_analysis import *
#from PIL import Image
#import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader

In [2]:
def model_dino_official_load(arch_name, patch_size, 
                             models_dir="../trained_model_weights/dino_official/backbone/"):
    # getting data file path
    weight_path = os.path.join(models_dir, f"dino_{arch_name}{patch_size}_pretrain.pth")
    checkpoint = torch.load(weight_path, map_location="cpu")

    if arch_name == "deitsmall":
        arch = "vit_small"
    elif arch_name == "vitbase":
        arch = "vit_base"

    model = vits.__dict__[arch](patch_size=patch_size)
    model.load_state_dict(checkpoint, strict=True)
        
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    for p in model.parameters():
        p.requires_grad = False
    model.eval()
    model.to(device)
    return model, device

In [3]:
def model_deit_official_load(arch="vit_small", patch_size=16):
    weight_path = "../trained_model_weights/deit_official/deit_small_patch16_224.pth"
    checkpoint = torch.load(weight_path, map_location="cpu")
    model = vits.__dict__[arch](patch_size=patch_size)
    model.load_state_dict(checkpoint["model"], strict=False) # ignore head weights
    
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    for p in model.parameters():
        p.requires_grad = False
    model.eval()
    model.to(device)
    return model, device

In [4]:
dataset_dir = "../dataset/Costela_Woods_2019/preprocessed_data/clips_png"
movie_info = np.load(f"{dataset_dir}/movie_info.npz", allow_pickle=True)

In [5]:
len(movie_info)

209

In [6]:
patch_size = 16
blur_size = patch_size * 2
batch_size = 32

transform = pth_transforms.Compose([
    pth_transforms.ToTensor(),
    pth_transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [7]:
save_dir = "../dataset/Costela_Woods_2019/preprocessed_data/vit_official_gaze_pos"

In [8]:
model_di, device = model_dino_official_load(arch_name="deitsmall", patch_size=16)
model_sl, _ = model_deit_official_load()

In [9]:
for movie_name in tqdm(movie_info.files):    
    print(movie_name)
    m_info = movie_info[movie_name].item()
    size = m_info["size"]
    resize = m_info["resize"]
    size_ratio = size / resize
    num_frame = m_info["num_frames"]
    
    
    image_path_list = sorted(glob.glob(f"{dataset_dir}/{movie_name}/*.png"))
    #assert len(image_path_list) == num_frame, "num frame is not matched."
    
    dataset = ImageDataset(image_path_list, transform)
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=False)
    
    res_dict = {}
    res_dict["info"] = ["num_models", "depth", "num_head+mean", "num_frames", "xy"]

    gaze_pos_model_di = get_gaze_pos_model_dataset(model_di, device, dataloader, patch_size, blur_size)
    gaze_pos_model_di *= size_ratio
    
    gaze_pos_model_sl = get_gaze_pos_model_dataset(model_sl, device, dataloader, patch_size, blur_size)
    gaze_pos_model_sl *= size_ratio

    res_dict[f"dino_deit_small16"] = gaze_pos_model_di
    res_dict[f"supervised_deit_small16"] = gaze_pos_model_sl
    np.savez_compressed(f"{save_dir}/{movie_name}.npz", **res_dict)

  0%|          | 0/209 [00:00<?, ?it/s]

ADVEN_12a_c


  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

ADVEN_15a_c


...

  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

WINGE_9a_c


  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]